# Load the modules

In [1]:
import cudf
from collections import OrderedDict
import numpy as np
import datetime as dt

In [2]:
print(cudf.__version__)

0.17.0


# Read the data

In [3]:
transactions_partitions_cnt = 10
transactions_path = 's3://bsql/data/seattle_parking/parking_MayJun2019.parquet/partition_idx={partition}/'
transactions_parq = [transactions_path.format(partition=p) for p in range(transactions_partitions_cnt)]

locations_parq = 's3://bsql/data/seattle_parking/parking_locations.parquet/'

In [10]:
transactions = cudf.read_parquet(transactions_parq, storage_options={'anon': True})
locations = cudf.read_parquet(locations_parq, storage_options={'anon': True})

In [11]:
print('The dataset has {0} records and {1} columns.'.format(*df.shape))

The dataset has 12168760 records and 9 columns.


In [12]:
print(df.columns)

Index(['OccupancyDateTime', 'PaidOccupancy', 'SourceElementKey', 'dow', 'year',
       'month', 'day', 'hour', 'minute'],
      dtype='object')


In [13]:
df.head().to_pandas()

,OccupancyDateTime,PaidOccupancy,SourceElementKey,dow,year,month,day,hour,minute
0,2019-05-24 08:35:00,1,35438,4,2019,5,24,8,35
1,2019-05-24 13:41:00,3,58402,4,2019,5,24,13,41
2,2019-05-24 09:29:00,3,59954,4,2019,5,24,9,29
3,2019-05-24 08:32:00,1,76961,4,2019,5,24,8,32
4,2019-05-24 15:17:00,1,32466,4,2019,5,24,15,17


# Extract date information

In [8]:
df['year'] = df['OccupancyDateTime']._column.year
df['month'] = df['OccupancyDateTime']._column.month
df['day'] = df['OccupancyDateTime']._column.day

df['hour'] = df['OccupancyDateTime']._column.hour
df['minute'] = df['OccupancyDateTime']._column.minute

df[['OccupancyDateTime','year','month','day','hour', 'minute']].head().to_pandas()

,OccupancyDateTime,year,month,day,hour,minute
0,2019-05-24 08:35:00,2019,5,24,8,35
1,2019-05-24 13:41:00,2019,5,24,13,41
2,2019-05-24 09:29:00,2019,5,24,9,29
3,2019-05-24 08:32:00,2019,5,24,8,32
4,2019-05-24 15:17:00,2019,5,24,15,17


In [13]:
counts = df.groupby(['year', 'month', 'day']).agg({'OccupancyDateTime': 'count'})
counts

OccupancyDateTime
year month day                   
2019 5     1               959957
           2               960058
           3               959713
           4               970780
           6               953855
           7               953749
           8               959130
           9               954661
           10              959962
           11              972658
           13              949417
           14              958798
           15              956968
           16              958204
           17              960651
           18              957570
           20              956194
           21              962393
           22              962377
           23              956228
           24              954530
           25              963938
           28              950981
           29              954106
           30              959746
           31              959143
     6     1               966546
           3               946295
           4               954473
           5               957349
           6               956514
           7               958960
           8               970800
           10              952197
           11              946510
           12              952578
           13              951078
           14              948080
           15              958350
           17              942704
           18              951444
           19              948173
           20              948713
           21              941868
           22              953226
           24              938273
           25              935960
           26              941543
           27              944252
           28              939084
           29              944302

time: 384 ms


In [14]:
print('Average number of transactions per day: {0:.0f}'.format(counts['OccupancyDateTime'].mean()))

Average number of transactions per day: 954413
time: 998 µs


# All parking locations

In [15]:
locations = df[['SourceElementKey', 'BlockfaceName', 'SideOfStreet',
       'ParkingTimeLimitCategory', 'ParkingSpaceCount',
       'PaidParkingArea', 'PaidParkingSubArea',
       'ParkingCategory', 'Location']].drop_duplicates()

locations.head().to_pandas()

,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,ParkingCategory,Location
4080,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,120,4.0,Pioneer Square,Core,Paid Parking,POINT (-122.33469356 47.6028728)
1336,1002,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,NE,120,8.0,Pioneer Square,Core,Paid Parking,POINT (-122.33451266 47.60294861)
4455,1006,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,NE,120,7.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.33514326 47.60367439)
1026,1009,1ST AVE BETWEEN MADISON ST AND SPRING ST,SW,30,5.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.3366575 47.60501765)
3208,1010,1ST AVE BETWEEN MADISON ST AND SPRING ST,NE,120,5.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.33644748 47.6051007)


time: 8.57 s


In [16]:
print('Number of parking locations in Seattle: {0}'.format(locations.shape[0]))

Number of parking locations in Seattle: 1528
time: 518 µs


In [17]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0]
    return lon.str.stod()

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1]
    return lon.str.stod()
    
locations['longitude'] = extractLon(locations['Location'])
locations['latitude'] = extractLat(locations['Location'])

locations[['Location', 'longitude', 'latitude']].head().to_pandas()

,Location,longitude,latitude
4080,POINT (-122.33469356 47.6028728),-122.334694,47.602873
1336,POINT (-122.33451266 47.60294861),-122.334513,47.602949
4455,POINT (-122.33514326 47.60367439),-122.335143,47.603674
1026,POINT (-122.3366575 47.60501765),-122.336658,47.605018
3208,POINT (-122.33644748 47.6051007),-122.336447,47.605101


time: 94.6 ms


# Average occupancy

In [18]:
def avgOccupancy(PaidOccupancy, ParkingSpaceCount, AvgOccupancy):
    for i, (paid, available) in enumerate(zip(PaidOccupancy, ParkingSpaceCount)):
        AvgOccupancy[i] = min(1.0, paid / available) # cap it at 100%, sometimes we see more paid occupancy than spaces available
        
df = (
    df[['OccupancyDateTime', 'PaidOccupancy', 'ParkingSpaceCount'
              , 'SourceElementKey', 'BlockfaceName', 'SideOfStreet'
              , 'ParkingTimeLimitCategory', 'ParkingSpaceCount'
              , 'PaidParkingArea', 'PaidParkingSubArea', 'ParkingCategory', 'dow', 'year', 'month'
              , 'day', 'hour', 'minute']]
    .apply_rows(
        avgOccupancy
        , incols=['PaidOccupancy', 'ParkingSpaceCount']
        , outcols={'AvgOccupancy': np.float64}
        , kwargs={}
    )
)
df.head()

,OccupancyDateTime,PaidOccupancy,ParkingSpaceCount,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,PaidParkingArea,PaidParkingSubArea,ParkingCategory,dow,year,month,day,hour,minute,AvgOccupancy
0,2019-05-24 08:35:00,1.0,4.0,35438,SENECA ST BETWEEN BOYLSTON AVE AND HARVARD AVE,SE,120,First Hill,UKN,Paid Parking,4,2019,5,24,8,35,0.25
1,2019-05-24 19:57:00,3.0,3.0,57862,REPUBLICAN ST BETWEEN QUEEN ANNE AVE N AND 1ST...,S,120,Uptown,Core,Paid Parking,4,2019,5,24,19,57,1.00
2,2019-05-24 11:21:00,1.0,4.0,53542,8TH AVE BETWEEN PINE ST AND OLIVE WAY,NE,120,Commercial Core,Retail,Paid Parking,4,2019,5,24,11,21,0.25
3,2019-05-24 11:56:00,2.0,4.0,9354,BLANCHARD ST BETWEEN 1ST AVE AND 2ND AVE,SE,120,Belltown,South,Paid Parking,4,2019,5,24,11,56,0.50
4,2019-05-24 10:57:00,13.0,7.0,35889,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,600,South Lake Union,North,Paid Parking,4,2019,5,24,10,57,1.00


time: 1.71 s


In [19]:
search_date_f = dt.datetime.strptime('2019-05-24T10:00:00', '%Y-%m-%dT%H:%M:%S')
search_date_t = dt.datetime.strptime('2019-05-24T10:59:59', '%Y-%m-%dT%H:%M:%S')
df.query('''SourceElementKey == 35889 and OccupancyDateTime >= @search_date_f and OccupancyDateTime <= @search_date_t'''
).sort_values(by='OccupancyDateTime').head(5).to_pandas()

,OccupancyDateTime,PaidOccupancy,ParkingSpaceCount,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,PaidParkingArea,PaidParkingSubArea,ParkingCategory,dow,year,month,day,hour,minute,AvgOccupancy
1454442,2019-05-24 10:00:00,13.0,7.0,35889,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,600,South Lake Union,North,Paid Parking,4,2019,5,24,10,0,1.0
1658508,2019-05-24 10:01:00,13.0,7.0,35889,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,600,South Lake Union,North,Paid Parking,4,2019,5,24,10,1,1.0
2273492,2019-05-24 10:02:00,13.0,7.0,35889,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,600,South Lake Union,North,Paid Parking,4,2019,5,24,10,2,1.0
636682,2019-05-24 10:03:00,13.0,7.0,35889,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,600,South Lake Union,North,Paid Parking,4,2019,5,24,10,3,1.0
2022358,2019-05-24 10:04:00,13.0,7.0,35889,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,600,South Lake Union,North,Paid Parking,4,2019,5,24,10,4,1.0


time: 4.57 s


In [20]:
def calcMean(AvgOccupancy, ParkingSpaceCount, MeanOccupancy):
    '''
        Calculate mean
    '''
    for i, (avgOccSum, avgCnt) in enumerate(zip(AvgOccupancy, ParkingSpaceCount)):
        MeanOccupancy[i] = float(avgOccSum) / avgCnt

df_agg_dt = (
    df
    .groupby(['SourceElementKey', 'dow','hour'])
    .agg({
          'ParkingSpaceCount': 'count'
        , 'AvgOccupancy': 'sum'
    })
    .reset_index()
)

df_agg_dt = df_agg_dt.apply_rows(
    calcMean
    , incols=['AvgOccupancy', 'ParkingSpaceCount']
    , outcols={'MeanOccupancy':np.float64}
    , kwargs={}
)

df_agg_dt.drop_column('AvgOccupancy')
df_agg_dt.drop_column('ParkingSpaceCount')

df_agg_dt.head().to_pandas()

,SourceElementKey,dow,hour,MeanOccupancy
0,1001,0,9,0.062500
1,1001,0,10,0.319643
2,1001,0,11,0.405357
3,1001,0,12,0.625595
4,1001,0,13,0.553571


time: 795 ms


# Find the best parking

In [21]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="todrabas_test")
location = geolocator.geocode("2300 7th Ave Seattle")

locations['LON_Ref'] = location.longitude
locations['LAT_Ref'] = location.latitude

time: 737 ms


In [22]:
from math import sin, cos, sqrt, atan2, pi

def calculateDistance(latitude, longitude, LAT_Ref, LON_Ref, Distance):
    R = 3958.8 # Earth's radius in miles
    
    for i, (lt, ln, lt_r, ln_r) in enumerate(zip(latitude, longitude, LAT_Ref, LON_Ref)):
        lt_rad = lt / 180.0 * pi
        ln_rad = ln / 180.0 * pi
        
        dlon = (ln_r - ln) / 180.0 * pi
        dlat = (lt_r - lt) / 180.0 * pi
        a = (sin(dlat/2.0))**2 + cos(lt_rad) * cos(lt_rad) * (sin(dlon/2.0))**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c
        Distance[i] = distance * 5280 # in feet
        
locations = locations.apply_rows(
    calculateDistance
    , incols=['latitude', 'longitude', 'LAT_Ref', 'LON_Ref']
    , outcols={'Distance':np.float64}
    , kwargs={}
)

locations.head().to_pandas()

,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,ParkingCategory,Location,longitude,latitude,LON_Ref,LAT_Ref,Distance
4080,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,120,4.0,Pioneer Square,Core,Paid Parking,POINT (-122.33469356 47.6028728),-122.334694,47.602873,-122.341789,47.617858,5738.588242
1336,1002,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,NE,120,8.0,Pioneer Square,Core,Paid Parking,POINT (-122.33451266 47.60294861),-122.334513,47.602949,-122.341789,47.617858,5725.999730
4455,1006,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,NE,120,7.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.33514326 47.60367439),-122.335143,47.603674,-122.341789,47.617858,5426.381931
1026,1009,1ST AVE BETWEEN MADISON ST AND SPRING ST,SW,30,5.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.3366575 47.60501765),-122.336658,47.605018,-122.341789,47.617858,4851.330142
3208,1010,1ST AVE BETWEEN MADISON ST AND SPRING ST,NE,120,5.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.33644748 47.6051007),-122.336447,47.605101,-122.341789,47.617858,4835.860012


time: 341 ms


In [23]:
# get only meters within 1000 ft
closest = locations.query('Distance < 1000')

closest = closest.merge(df_agg_dt, how='inner', on=['SourceElementKey']).query('dow == 3 and hour == 13')
closest = closest.sort_values(by='MeanOccupancy')

closest_host = closest[['BlockfaceName', 'SideOfStreet',
       'ParkingTimeLimitCategory', 'ParkingSpaceCount', 'PaidParkingArea',
       'PaidParkingSubArea', 'ParkingCategory', 'Location', 'Distance', 'dow', 'hour', 'MeanOccupancy', 'longitude', 'latitude']].head().to_pandas()
closest_host

,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,ParkingCategory,Location,Distance,dow,hour,MeanOccupancy,longitude,latitude
2848,BELL ST BETWEEN 7TH AVE AND 8TH AVE,NW,240,8.0,Denny Triangle,North,Paid Parking,POINT (-122.34098431 47.61773608),202.749256,3,13,0.000000,-122.340984,47.617736
2115,BELL ST BETWEEN 6TH AVE AND 7TH AVE,SE,240,5.0,Denny Triangle,North,Paid Parking,POINT (-122.34168549 47.61701942),306.872107,3,13,0.008333,-122.341685,47.617019
2315,DEXTER AVE N BETWEEN JOHN ST AND THOMAS ST,W,120,7.0,South Lake Union,South,Paid Parking,POINT (-122.3425489 47.62030601),912.531672,3,13,0.038690,-122.342549,47.620306
2815,6TH AVE BETWEEN BATTERY ST AND WALL ST,NE,240,6.0,Denny Triangle,North,Paid Parking,POINT (-122.34401361 47.61786447),547.114925,3,13,0.066667,-122.344014,47.617864
2152,JOHN ST BETWEEN 8TH AVE N AND 9TH AVE N,S,600,21.0,South Lake Union,South,Paid Parking,POINT (-122.34052184 47.6196361),719.717235,3,13,0.114815,-122.340522,47.619636


time: 337 ms


# Plot the parking spots on the map

We're using gmaps python package that can be found here: https://github.com/pbugnion/gmaps. Follow the instructions contained within the README.md about how to install the package so the map shows properly in jupyter lab.

In [24]:
closest_host[['BlockfaceName', 'Distance', 'MeanOccupancy']].to_dict('records')
info_box_template = """
<dl>
<dt>Name</dt><dd>{BlockfaceName}</dd>
<dt>Distance</dt><dd>{Distance:.0f}</dd>
<dt>Occupancy (AVG)</dt><dd>{MeanOccupancy:.3f}</dd>
</dl>
"""

parking_info = [info_box_template.format(**parking) for parking in closest_host[['BlockfaceName', 'Distance', 'MeanOccupancy']].to_dict('records')]

time: 4.77 ms


In [25]:
closest_host.to_dict('records')[0]['latitude']

47.61773607999999

time: 5.76 ms


In [26]:
import gmaps
import gmaps.datasets
gmaps.configure(api_key="--->>> YOUR KEY <<<---") # Your Google API key, go to https://console.developers.google.com

parking_layer = gmaps.symbol_layer(
    closest_host[['latitude', 'longitude']], fill_color="green", stroke_color="green", scale=3, info_box_content=parking_info
)

destinations_layer = gmaps.symbol_layer(
    [[location.latitude, location.longitude]]
    , info_box_content=['DESTINATION']
    , scale=5
    , fill_color="red"
    , stroke_color="red"
)

parkings = closest_host.to_dict('records')

lines_layer = gmaps.drawing_layer(features=[
    gmaps.Line(
        start= (parking['latitude'], parking['longitude'])
        , end = (location.latitude, location.longitude)
        , stroke_weight=2
        , stroke_color="red"
    )
    for parking in parkings]
)

fig = gmaps.figure(layout={'height': '500px'})
fig.add_layer(parking_layer)
fig.add_layer(destinations_layer)
fig.add_layer(lines_layer)
fig

Figure(layout=FigureLayout(height='500px'))

time: 56.5 ms
